In [1]:
# this is only necessary if the jpype module (imported by efmtool_link.efmtool_intern) is not able to find Java on its own
import os
os.environ['JAVA_HOME'] = r"C:\\Program Files\\AdoptOpenJDK\\jdk-11.0.8.10-hotspot"

In [2]:
import efmtool_link.efmtool_intern as efmtool_intern
import numpy
import cobra
model = cobra.io.read_sbml_model(r"metatool_example_no_ext.xml")


No objective coefficients in model. Unclear what should be optimized


In [3]:
# calculate nullspace using rational arithmetic
st = cobra.util.array.create_stoichiometric_matrix(model, array_type='lil')
kn = efmtool_intern.null_rat_efmtool(st)
numpy.max(numpy.abs(st @ kn))

0.0

In [4]:
# calculate elementary modes of the network
import efmtool_link.efmtool_extern as efmtool_extern
import efmtool_link.efmtool4cobra as efmtool4cobra
reversible, irrev_backwards_idx = efmtool4cobra.get_reversibility(model)
# IMPORTANT: an irreversible reaction MUST point in the forward reaction; if this is not the case
# flip the reactions in irrev_backwards_idx (none in the running example) before calculating modes
st[:, irrev_backwards_idx] *= -1
efms = efmtool_extern.calculate_flux_modes(st, reversible)
# flip back to the original reaction directions (does nothing in the running example):
st[:, irrev_backwards_idx] *= -1
efms[irrev_backwards_idx, :] *= -1
print(efms.shape[1], "elementary modes.") # each column in efms represents a mode
numpy.max(numpy.abs(st @ efms)) # should be a value close to 0


2021-03-26  13:26:01.322  main                     INFO     | =====================================================
2021-03-26  13:26:01.322  main                     INFO     | efmtool version 4.7.1, 2009-12-04 18:30:05
2021-03-26  13:26:01.326  main                     INFO     | Copyright (c) 2009, Marco Terzer, Zurich, Switzerland
2021-03-26  13:26:01.330  main                     INFO     | This is free software, !!! NO WARRANTY !!!
2021-03-26  13:26:01.333  main                     INFO     | See LICENCE.txt for redistribution conditions
2021-03-26  13:26:01.335  main                     INFO     | =====================================================
2021-03-26  13:26:01.527  main    efm.impl         INFO     | Elemetary flux mode computation
2021-03-26  13:26:01.527  main    efm.impl         INFO     | Implementation:
2021-03-26  13:26:01.527  main    efm.impl         INFO     | ..algorithm name   : SequentialDoubleDescriptionImpl
2021-03-26  13:26:01.527  main    efm.impl     

6.661338147750939e-16

In [5]:
# network compression (currently combination of reaction subsets only)
# IMPORTANT: the model is modified by this function, if you want to keep the full model copy it first
subT = efmtool4cobra.compress_model_sympy(model) # subT is a matrix for conversion of flux vectors between the full and compressed model
rd = cobra.util.array.create_stoichiometric_matrix(model, array_type='lil')
# model compression makes sure that irreversible reactions always point in the forward direction
rev_rd = [int(r.reversibility) for r in model.reactions]

0.030999999999039574
0.01599999999962165


In [6]:
# calculate modes in the compressed network
rd_efms = efmtool_extern.calculate_flux_modes(rd, rev_rd)
print(rd_efms.shape[1], "elementary modes.")
# subT @ rd_efms are the same modes as efms (but can be in different order and scaled differently)
numpy.max(numpy.abs(st @ subT @ rd_efms)) # should be a value close to 0

2021-03-26  13:26:02.521  main                     INFO     | =====================================================
2021-03-26  13:26:02.522  main                     INFO     | efmtool version 4.7.1, 2009-12-04 18:30:05
2021-03-26  13:26:02.522  main                     INFO     | Copyright (c) 2009, Marco Terzer, Zurich, Switzerland
2021-03-26  13:26:02.522  main                     INFO     | This is free software, !!! NO WARRANTY !!!
2021-03-26  13:26:02.522  main                     INFO     | See LICENCE.txt for redistribution conditions
2021-03-26  13:26:02.522  main                     INFO     | =====================================================
2021-03-26  13:26:02.693  main    efm.impl         INFO     | Elemetary flux mode computation
2021-03-26  13:26:02.694  main    efm.impl         INFO     | Implementation:
2021-03-26  13:26:02.694  main    efm.impl         INFO     | ..algorithm name   : SequentialDoubleDescriptionImpl
2021-03-26  13:26:02.694  main    efm.impl     

6.661338147750939e-16